# Storage

The storage is a mapping from a `key` to a `value`. The number of keys is for all practical purposes infinite. The value can be up to 32 bytes. Every value is initialized with a 0.

This is like the SSD in your computer. Storage is non-volatile.

![title](../static/storage.png)

source: https://docs.alchemy.com/docs/smart-contract-storage-layout

We will represent the storage as a dictionary.

In [ ]:
use std::collections::HashMap;

struct KeyValue {
    storage: HashMap<String, String>,
}

impl KeyValue {
    fn new() -> Self {
        KeyValue {
            storage: HashMap::new(),
        }
    }

    fn load(&self, key: &str) -> Option<&String> {
        self.storage.get(key)
    }

    fn store(&mut self, key: String, value: String) {
        self.storage.insert(key, value);
    }
}


### Warm/Cold

It costs different amount of gas whether we access a warm or cold storage slot. 
A slot is said to be `warm` if it was access before. Otherwise it is `cold`.
Accessing a slot that is `cold` costs more gas than accessing a `warm` slot.

We implement that logic by keeping track of a `cache`. When we load a storage slot we save its key in that `cache`. If a key is in that `cache` it is said to be warm.

In [ ]:
use std::collections::HashMap;

struct Storage {
    storage: HashMap<String, u8>,
    cache: Vec<String>,
}

impl Storage {
    fn new() -> Self {
        Storage {
            storage: HashMap::new(),
            cache: vec![],
        }
    }

    fn load(&mut self, key: &str) -> (bool, u8) {
        let warm = self.cache.contains(&key.to_string());

        if !warm {
            self.cache.push(key.to_string());
        }

        let value = self.storage.get(key).copied().unwrap_or(0x00);
        (warm, value)
    }

    fn store(&mut self, key: String, value: u8) {
        self.storage.insert(key, value);
    }
}


In [ ]:
let mut storage = Storage::new();

We store `420` in storage slot `1`

In [ ]:
storage.store(1u64.to_string(), 420);

Notice how the first time retrieving something from storage slot `1` its `cold`

In [ ]:
let (warm, value) = storage.load(&1u64.to_string());

(False, 420)

Now storage slot `1` is `warm`

In [ ]:
let (warm, value) = storage.load(&1.to_string());

(True, 420)

Reading a random storage that was not set to any value will return 0 and **not** throw an exception.

In [35]:
storage.load(42069)

0